In [1]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely import geometry
import random
import math
%matplotlib inline

In [2]:
data = pd.read_csv('boundingbox.csv')
data['lon direction']=data['\ufefflon direction'] 
del data['\ufefflon direction']

In [3]:
brunei = [5.033182, 115.086013]
phillipines = [14.790988, 120.242715]
singapore = [1.461519, 103.831384]
vietnam = [11.901643, 109.170094]
bases = [brunei,phillipines,singapore,vietnam]

In [4]:
def degMinSecToDecimal(deg, mn, sec):
    """ convert (degrees,minutes,seconds) to decimal-degrees """
    decimal = deg + (mn/60.0) + (sec/3600.0)
    return decimal

def calc_distance(lat1, lon1, lat2, lon2):
    '''This function calculates the distance between two coordinates. It takes 2 lat/lon pairs of coordinates in 
    decimal representation and first converts them to radians. The coordinates must already be in there decimal 
    representation to be used as inputs. It then calculates the distance between the two coordinates using the
    great circle distance equation from the reference. From there it converts that distance from radians back to nm.'''
    rlat1 = deg2rad(lat1) #converts lat/longs from degrees to radians
    rlon1 = deg2rad(lon1)
    rlat2 = deg2rad(lat2)
    rlon2 = deg2rad(lon2)
    d = math.acos(math.sin(rlat1) * math.sin(rlat2) + math.cos(rlat1) * math.cos(rlat2) * math.cos(rlon1 - rlon2))
    # calculates distance between two points in radians. formula provided in reference
    nm = rad2nm(d) # this converts the distance from radians to nm
    return nm
#end of calc_distance

def rad2nm (radians):
    '''This function converts a distance in radians into nautical miles. It will take one input in radians.'''
    nm = ((180 * 60) / math.pi) * radians
    return nm #returns the distance in nm
#end of rad2nm


def deg2rad (degrees):
    '''This function will convert a coordinate location in decimal representation in degrees to its value
    in radians. The input is the decimal degree location of a single coordinate.'''
    radians = degrees * ((math.pi)/180)
    return radians #returns the value in radians
# end of deg2rad

In [5]:
boxpoints =[]
for index,row in data.iterrows():
    lat = degMinSecToDecimal(row['lat degrees '],row['lat minutes '],row['lat seconds'])
    lon = degMinSecToDecimal(row['lon degrees'],row['lon minutes'],row['lon seconds'])
    if row['lat direction']=='S':
        lat = -lat
    if row['lon direction']=='W':
        lon= -lon
    boxpoints.append([lat,lon])
    
poly = geometry.Polygon(boxpoints)
#poly.wkt

In [6]:
def get_random_point_in_polygon(poly):
    (minx, miny, maxx, maxy) = poly.bounds
    test = False
    while test == False:
        p = geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p):
            #print(p)
            test = True
            return p

In [7]:
def initial_resupply_locations(rs_ship,rs_ship_dict,maxdistance):
    test=False
    while test == False:
        point = get_random_point_in_polygon(poly)
        loc = calc_distance(point.x,point.y,rs_ship_dict[rs_ship][5][0],rs_ship_dict[rs_ship][5][1])
        if loc <= maxdistance:
            test=True
            goodpoint=point
    return goodpoint

In [8]:
def resupply_back_out(rs,rs_ship_dict,maxdistance,speed):
    test=False
    while test == False:
        point = get_random_point_in_polygon(poly)
        distance = calc_distance(point.x,point.y,rs_ship_dict[rs][5][0],rs_ship_dict[rs][5][1])
        if distance <= maxdistance:
            test=True
            goodpoint=point
            time=distance/speed
    return time,goodpoint

In [9]:
def resupply_back_out2(patrol,ship_dict,rs,rs_ship_dict,maxdistance,speed):
    test=False
    while test == False:
        point = get_random_point_in_polygon(poly)
        distance = calc_distance(point.x,point.y,rs_ship_dict[rs][5][0],rs_ship_dict[rs][5][1])
        if distance <= maxdistance:
            test=True
            goodpoint=point
            bhdistance = calc_distance(goodpoint.x,goodpoint.y,ship_dict[patrol][4].x,ship_dict[patrol][4].y)
            time=bhdistance/speed
    return time,goodpoint

In [10]:
def patrol_needs_fuel(ship,ship_dict,rs_ship_dict,avail_rs_list,speed):
    ship_point = ship_dict[ship][4]
    distance_list = []
    for rs_ship in avail_rs_list:
        rs_loc = rs_ship_dict[rs_ship][4]
        distance_list.append([rs_ship,calc_distance(ship_point.x,ship_point.y,rs_loc.x,rs_loc.y)])
    rs,dist = find_min_distance(distance_list)
    time = dist/speed
    return rs,time

In [11]:
def find_distancelist(ship,ship_dict,rs_ship_dict,avail_rs_list):
    ship_point = ship_dict[ship][4]
    distance_list = []
    fuel_need = ship_dict[ship][2]-ship_dict[ship][0]
    for rs_ship in avail_rs_list:
        if rs_ship_dict[rs_ship][0] >= fuel_need:
            rs_loc = rs_ship_dict[rs_ship][4]
            distance_list.append([rs_ship,calc_distance(ship_point.x,ship_point.y,rs_loc.x,rs_loc.y)])
    return distance_list

In [12]:
def find_min_distance(distancelist,speed):
    min_distance = 1000000
    #min_ship = line[0]
    for line in distancelist:
        if line[1] < min_distance:
            min_distance = line[1]
            min_ship = line[0]
            time = min_distance/speed
    return min_ship,time,min_distance

In [13]:
def patrol_back_out(patrol_ship,ship_dict,maxdistance,speed):
    test=False
    while test == False:
        point = get_random_point_in_polygon(poly)
        distance = calc_distance(point.x,point.y,ship_dict[patrol_ship][4].x,ship_dict[patrol_ship][4].y)
        if distance <= maxdistance:
            test=True
            goodpoint=point
            time = distance/speed
    return goodpoint,time


In [14]:
def resupply_ship_needs_fuel(patrol_ship,ship_dict,rs,rs_ship_dict,speed):
    point= ship_dict[patrol_ship][4]
    distance = calc_distance(point.x,point.y,rs_ship_dict[rs][5][0],rs_ship_dict[rs][5][1])
    time = distance/speed
    return time

In [15]:
point=get_random_point_in_polygon(poly)
#print(point)

In [16]:
from queue import PriorityQueue
import abc

class SimpleKit:
    """
    To create a SimpleKit model:
        Your model class must be a subclass of SimpleKit.
        Your constructor (__init__()) must call SimpleKit.__init__(self).
        You must override the abstract init() method to start your model.
    Note:
        Your model cannot implement methods named run(), schedule(), or halt().
    """
    __metaclass__ = abc.ABCMeta

    def __init__(self):
        """Initialize a pending events list and set model_time to 0."""
        self.event_list = PriorityQueue()
        self.model_time = 0.0
    
    @abc.abstractmethod
    def init(self):
        """This abstract method must be overridden in your model class."""

    def run(self):
        """Execute the model logic."""
        self.init()
        while not self.event_list.empty():
            event_notice = self.event_list.get()
            self.model_time = event_notice[0]
            event_notice[1](*event_notice[2:])

    def schedule(self, event, delay, *args):
        """Add an event to the pending events.

        Args:
            event: The name of the event method to be scheduled.
            delay: The amount of model time by which to delay the execution.
            args: (optional) Any arguments required by the event.
        """
        if delay < 0:
            raise RuntimeError('Negative delay is not allowed.')
        event_notice = [delay + self.model_time, event] + list(args)
        self.event_list.put(event_notice)
        print(self.event_list.values)
    
    def halt(self):
        """Terminate a simulation run by clearing the pending events list."""
        while not self.event_list.empty():
            self.event_list.get()

In [34]:
from simplekit import SimpleKit
import numpy as numpy
import math
import csv

class Logistic_Model(SimpleKit):
    """Implementation of a Maritime Logistics System for refueling ."""

    def __init__(self, outfile):
        """Construct an instance of the M/M/k."""
        SimpleKit.__init__(self)        
        
        #initialize all values of self for the system
        #Demand values
        self.CSG_cap_fuel = 17000+2200*3+2100
        self.DDG_cap_fuel = 2629
        self.CG_cap_fuel = 3758
        #self.CVN_cap_ammo = 0
        self.CSG_cap_ammo = 268
        self.DDG_cap_ammo = 26
        self.CG_cap_ammo = 18
        
        #Resupply Ship Values
            #FCS:
        self.TAOE_cap_fuel = 177000 #barrels
        self.TAOE_cap_ammo = 1800 #tons
            
            #Lewis and Clark:
        self.TAKE_cap_fuel = 23450 #barrels
        self.TAKE_cap_ammo = 6000 #tons 
            
            #CLS
        self.CLS5000_cap_fuel = 2900 #barrels
        self.CLS5000_cap_ammo = 500  #tons
        
        #self.speed = 18 #knots
        #create and output file based on the given outfile name
        b = open(outfile, 'w', newline='') 
        fieldnames = ['Model_Time', 'Event', "Ship", "Ship Fuel Qty",'Ship Ammo Qty',
                      "Resupply Ship", "Resupply Fuel Qty",'Resupply Ammo Qty']
        self.outcsvfile = csv.DictWriter(b, fieldnames=fieldnames)
        self.outcsvfile.writeheader()
     
    
    def init(self):
        """Initialize all state variables, schedule first demand."""
        
        #Demand ships
        self.ship_list = ['CSG', #carrier
                          'DDG_62', 'DDG_63', 'DDG_85', 'DDG_89', #destroyers
                          'CG_62', 'CG_67'] #cruisers
        
                         #'ship'  : [fuel level,        ammo level,        fuel capacity,     ammo capacity     , location(tuple)]     
        self.ship_dict = {'CSG': [self.CSG_cap_fuel, self.CSG_cap_ammo, self.CSG_cap_fuel, self.CSG_cap_ammo, get_random_point_in_polygon(poly)], 
                          #'DDG_52': [self.DDG_cap_fuel, self.DDG_cap_ammo, self.DDG_cap_fuel, self.DDG_cap_ammo, get_random_point_in_polygon(poly)],
                          #'DDG_54': [self.DDG_cap_fuel, self.DDG_cap_ammo, self.DDG_cap_fuel, self.DDG_cap_ammo, get_random_point_in_polygon(poly)],
                          #'DDG_56': [self.DDG_cap_fuel, self.DDG_cap_ammo, self.DDG_cap_fuel, self.DDG_cap_ammo, get_random_point_in_polygon(poly)],
                          'DDG_62': [self.DDG_cap_fuel, self.DDG_cap_ammo, self.DDG_cap_fuel, self.DDG_cap_ammo, get_random_point_in_polygon(poly)],
                          'DDG_63': [self.DDG_cap_fuel, self.DDG_cap_ammo, self.DDG_cap_fuel, self.DDG_cap_ammo, get_random_point_in_polygon(poly)],
                          'DDG_85': [self.DDG_cap_fuel, self.DDG_cap_ammo, self.DDG_cap_fuel, self.DDG_cap_ammo, get_random_point_in_polygon(poly)],
                          'DDG_89': [self.DDG_cap_fuel, self.DDG_cap_ammo, self.DDG_cap_fuel, self.DDG_cap_ammo, get_random_point_in_polygon(poly)],
                          #'CG_54':  [self.CG_cap_fuel,  self.CG_cap_ammo,  self.CG_cap_fuel,  self.CG_cap_ammo, get_random_point_in_polygon(poly)],
                          'CG_62':  [self.CG_cap_fuel,  self.CG_cap_ammo,  self.CG_cap_fuel,  self.CG_cap_ammo, get_random_point_in_polygon(poly)],
                          'CG_67':  [self.CG_cap_fuel,  self.CG_cap_ammo,  self.CG_cap_fuel,  self.CG_cap_ammo, get_random_point_in_polygon(poly)]} 
        
        #Resupply ships:
        self.rs_ship_list = ['T_AOE_6', 'T_AKE_4']#,'T_AOE_8', 'T_AKE_3', ] 
                             #'CLS_1','CLS_2','CLS_3','CLS_4'] 
        
                            #'ship'    : [fuel level,        ammo level,        fuel capacity,     ammo capacity]
        self.rs_ship_dict = {'T_AOE_6': [self.TAOE_cap_fuel, self.TAOE_cap_ammo, self.TAOE_cap_fuel, self.TAOE_cap_ammo, [0,0],brunei],
                             #'T_AOE_8': [self.TAOE_cap_fuel, self.TAOE_cap_ammo, self.TAOE_cap_fuel, self.TAOE_cap_ammo, [0,0],phillipines],
                             #'T_AKE_3': [self.TAKE_cap_fuel, self.TAKE_cap_ammo, self.TAKE_cap_fuel, self.TAKE_cap_ammo, [0,0],brunei],
                             'T_AKE_4': [self.TAKE_cap_fuel, self.TAKE_cap_ammo, self.TAKE_cap_fuel, self.TAKE_cap_ammo, [0,0],vietnam]}
                             #'CLS_1': [self.CLS5000_cap_fuel, self.CLS5000_cap_ammo, self.CLS5000_cap_fuel, self.CLS5000_cap_ammo, [0,0],vietnam],
                             #'CLS_2': [self.CLS5000_cap_fuel, self.CLS5000_cap_ammo, self.CLS5000_cap_fuel, self.CLS5000_cap_ammo, [0,0],phillipines],
                             #'CLS_3': [self.CLS5000_cap_fuel, self.CLS5000_cap_ammo, self.CLS5000_cap_fuel, self.CLS5000_cap_ammo, [0,0],brunei],
                             #'CLS_4': [self.CLS5000_cap_fuel, self.CLS5000_cap_ammo, self.CLS5000_cap_fuel, self.CLS5000_cap_ammo, [0,0],singapore]}
        
        #resupply ships available to resupply
        self.avail_rs_list = ['T_AOE_6', 'T_AKE_4']#,'T_AOE_8', 'T_AKE_3', ] 
                             #'CLS_1','CLS_2','CLS_3','CLS_4'] 
        
        self.schedule(self.halt, 24*60)  #run the model for 15 days
        
        for ship in self.ship_list: #schedule when all ships run out of fuel and where they will be located 
            self.schedule(self.demand, random.uniform(96,144), ship)
        
        for rs_ship in self.rs_ship_list:
            self.rs_ship_dict[rs_ship][4] = initial_resupply_locations(rs_ship,self.rs_ship_dict,300)

        self.dumpState("Init", 'NA', 'NA', 'NA', 'NA','NA','NA')
    
    
    def demand(self,ship):
        """Check to determine if any ships need fuel/ammo"""
        self.ship_dict[ship][0]=0
        
        if len(self.avail_rs_list) == 0:
            self.schedule(self.demand, 1, ship) #no resupply ship is available, check back in an hour
            self.dumpState('!!Resupply Unavailable!!',ship,'NA','NA','NA','NA','NA') #surface ship can not be resupplied at the time
        else:
            dl = find_distancelist(ship,self.ship_dict,self.rs_ship_dict,self.avail_rs_list)
            if len(dl)== 0:
                self.schedule(self.demand, 1, ship) #no resupply ship is available, check back in an hour
                self.dumpState('!!Not big enough!!',ship,'NA','NA','NA','NA','NA')#surface ship can not be resupplied at the time
            else:
                rs,time,dist=find_min_distance(dl,18)
                self.avail_rs_list.remove(rs)
                self.schedule(self.resupply,time,ship,rs,dist)
                self.dumpState('Fuel Needed',ship,self.ship_dict[ship][0],self.ship_dict[ship][1],
                               rs,self.rs_ship_dict[rs][0],self.rs_ship_dict[rs][1])
                
    def resupply(self, ship, resupply,distance): 
        """Subtract fuel from resupply ship, reset fuel for surface ship, schedule both ships to go on patrol"""
        survival = random.uniform(0,1)
        if survival > 0.98:
            self.schedule(self.demand,1,ship)
            self.rs_ship_list.remove(resupply)
            self.dumpState('Resupply ship did not survive!!!!',ship,'NA','NA',resupply,'NA','NA')
        else:
            self.schedule(self.patrol, numpy.random.normal(4, 1), ship, resupply) 
            self.rs_ship_dict[resupply][1] -= self.ship_dict[ship][3] #subtract ammo from resupply
            self.rs_ship_dict[resupply][0] = self.rs_ship_dict[resupply][0] - self.ship_dict[ship][2] #subtract the fuel transferred from resupply ship
            self.ship_dict[ship][0] = self.ship_dict[ship][2] #reset the fuel guage for ship
            self.ship_dict[ship][1] = self.ship_dict[ship][3] #reset ammo for ship
            self.dumpState("Resupply Under Way", ship, self.ship_dict[ship][0],self.ship_dict[ship][1],
                           resupply, self.rs_ship_dict[resupply][0], self.rs_ship_dict[resupply][1])     
    
    def patrol(self, ship, resupply):
        """Schedule when the ship needs fuel and it's location at that time, check to see if the resupply ship needs fuel"""
        self.ship_dict[ship][4],time = patrol_back_out(ship,self.ship_dict,2000,18)
        
        self.schedule(self.demand, (random.uniform(48,72))+time, ship) #Schedule when the ship runs out of fuel!!! dictionary value based on avearge fuel rates 
        
        
        if (self.rs_ship_dict[resupply][0] <= 0.25*self.rs_ship_dict[resupply][2]) or (self.rs_ship_dict[resupply][1] <= 268): #check if the resupply ship needs fuel
            time = resupply_ship_needs_fuel(ship,self.ship_dict,resupply,self.rs_ship_dict,20)
            self.schedule(self.resupply_resupply, time, resupply) #schedule a time to resupply the resupply ship
            self.dumpState("Return to Base", 'NA', 'NA','NA', 
                           resupply, self.rs_ship_dict[resupply][0], self.rs_ship_dict[resupply][1])  
            self.dumpState("Patrolling", ship, self.ship_dict[ship][0],self.ship_dict[ship][1],
                           resupply, self.rs_ship_dict[resupply][0], self.rs_ship_dict[resupply][1])      
    
        else:
            #self.avail_rs_list.append(resupply)
            #resupply ship goes back to AO
            time,self.rs_ship_dict[resupply][4]=resupply_back_out2(ship,self.ship_dict,resupply,self.rs_ship_dict,300,20)
            self.schedule(self.resupply_available,time,resupply)
            self.dumpState("Patrolling", ship, self.ship_dict[ship][0],self.ship_dict[ship][1],
                           resupply, self.rs_ship_dict[resupply][0], self.rs_ship_dict[resupply][1])      
    
    def resupply_resupply(self, resupply):
        """Reset the resupply levels of the resupply ships, generate a location for it to travel to, append it to the available list"""
        self.rs_ship_dict[resupply][0] = self.rs_ship_dict[resupply][2] #resupply ship is topped back off
        self.rs_ship_dict[resupply][1] = self.rs_ship_dict[resupply][3] #ammo topped off
        time,self.rs_ship_dict[resupply][4] = resupply_back_out(resupply,self.rs_ship_dict,300,20)
        self.schedule(self.resupply_available,time,resupply)
        self.dumpState("Resupplying Resupply", "NA", "NA",'NA',
                       resupply, self.rs_ship_dict[resupply][0], self.rs_ship_dict[resupply][1])    
    def resupply_available(self,resupply):
        self.avail_rs_list.append(resupply)
        self.dumpState('Resupply Available','NA','NA','NA',resupply,'NA','NA')
        
    def dumpState(self, event, ship, ship_fuel, ship_ammo, resupply, resupply_fuel,resupply_ammo): #this state prints all values to the .csv file
        """Dump of the current state of the model."""
        self.outcsvfile.writerow({'Model_Time': float(self.model_time), 'Event': event, 
                                  'Ship':ship, 'Ship Fuel Qty': ship_fuel,'Ship Ammo Qty': ship_ammo, 
                                  'Resupply Ship': resupply,  'Resupply Fuel Qty': resupply_fuel,
                                  'Resupply Ammo Qty':resupply_ammo}) 
        print("Time:", self.model_time, "Event:", event,'Ship', ship, "Ship Fuel Qty:", ship_fuel,'Ship Ammo Qty:',ship_ammo,
              'Resupply Ship', resupply, "Resupply Fuel Qty:", resupply_fuel, 'Resupply Ammo Qty:',resupply_ammo)

In [35]:
Logistic_Model('Case2_Scenario3.csv').run()

Time: 0.0 Event: Init Ship NA Ship Fuel Qty: NA Ship Ammo Qty: NA Resupply Ship NA Resupply Fuel Qty: NA Resupply Ammo Qty: NA
Time: 103.47713075742456 Event: Fuel Needed Ship CG_62 Ship Fuel Qty: 0 Ship Ammo Qty: 18 Resupply Ship T_AOE_6 Resupply Fuel Qty: 177000 Resupply Ammo Qty: 1800
Time: 106.7777862135506 Event: Fuel Needed Ship DDG_89 Ship Fuel Qty: 0 Ship Ammo Qty: 26 Resupply Ship T_AKE_4 Resupply Fuel Qty: 23450 Resupply Ammo Qty: 6000
Time: 108.66734125586899 Event: !!Resupply Unavailable!! Ship DDG_62 Ship Fuel Qty: NA Ship Ammo Qty: NA Resupply Ship NA Resupply Fuel Qty: NA Resupply Ammo Qty: NA
Time: 109.39663576987684 Event: Resupply Under Way Ship CG_62 Ship Fuel Qty: 3758 Ship Ammo Qty: 18 Resupply Ship T_AOE_6 Resupply Fuel Qty: 173242 Resupply Ammo Qty: 1782
Time: 109.66734125586899 Event: !!Resupply Unavailable!! Ship DDG_62 Ship Fuel Qty: NA Ship Ammo Qty: NA Resupply Ship NA Resupply Fuel Qty: NA Resupply Ammo Qty: NA
Time: 110.66734125586899 Event: !!Resupply Una

In [36]:
335*8.5

2847.5